In [1]:
import pandas as pd

students = pd.read_csv('/Users/olgazdanova/Downloads/student.csv')
students

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences,grade
0,18,4,4,2,2,0,4,3,4,1,1,3,4,1
1,17,1,1,1,2,0,5,3,3,1,1,3,2,1
2,15,1,1,1,2,0,4,3,2,2,3,3,6,1
3,15,4,2,1,3,0,3,2,2,1,1,5,0,2
4,16,3,3,1,2,0,4,3,2,1,2,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,19,2,3,1,3,1,5,4,2,1,2,5,4,1
645,18,3,1,1,2,0,4,3,4,1,1,1,4,2
646,18,1,1,2,2,0,1,1,1,1,1,5,6,1
647,17,3,1,2,1,0,2,4,5,3,4,2,6,1


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
X = students.drop('grade', axis=1)
y = students['grade']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [3]:
logR = LogisticRegression()
logR.fit(X_train, y_train)
print(logR.coef_)

[[ 0.18371165  0.46279229  0.02488828 -0.14084635  0.38231793 -1.33099987
   0.0031495   0.05123498 -0.01806676 -0.66531288  0.02035118 -0.13587078
  -0.05586459]]


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(X_test_scaled.min())
print(X_test_scaled.max())

-3.0980006863408747
6.339516485831038


In [8]:
X_test_scaled[0].max()

np.float64(1.5719507530633017)

In [14]:
# продолжаем работать с масштабированными данными 
from sklearn.metrics import accuracy_score # импортируем метрику

# обучим модель логистической регрессии с базовыми параметрами на стандартизированных данных
model = LogisticRegression() # инициализируем модель
model.fit(X_train_scaled, y_train) # обучаем модель

y_train_pred = model.predict(X_train_scaled) # делаем предсказания на выборках
y_test_pred = model.predict(X_test_scaled)

# посчитаем метрику точности на тренировочной и тестовой выборках
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(train_accuracy, test_accuracy)

0.7572254335260116 0.7846153846153846


In [16]:
# теперь будем работать с НЕмасшабируемыми данными
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

# сделаем пайплайн, куда положим модель полиномизации признаков до 4 степени, модель масштабирования данных, логистическую регрессию без регуляризации
pipeline = Pipeline(
    [
        ('polinom', PolynomialFeatures(degree=4)),
        ('scaler', StandardScaler()),
        ('logR', LogisticRegression(penalty=None))
    ]
)

pipeline.fit(X_train, y_train) # обучаем пайплайн на тренировочной выборке

y_train_pred = pipeline.predict(X_train) # делаем предсказания 
y_test_pred = pipeline.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred) # выводим точность предсказания по выборкам
test_accuracy = accuracy_score(y_test, y_test_pred)

print(train_accuracy, test_accuracy)


0.9479768786127167 0.6846153846153846


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
from sklearn.model_selection import GridSearchCV
# добавим в пайплайн кол-во итераций для логистической регрессии и регуляризацию (чтобы убрать переобучение)
pipeline = Pipeline(
    [
        ('polinom', PolynomialFeatures(degree=4)),
        ('scaler', StandardScaler()),
        ('logreg', LogisticRegression(penalty='l2', max_iter=600))
    ]
)
# сделаем перебор параметра С в логистической регрессии, который из себя представляет 1/коэффициент регуляризации 
param_grid = {
    'logreg__C': [0.0001, 0.001, 0.01, 0.1, 1] # будем перебирать из этого списка
}

# важно чтобы имя шага и имя параметра назывались одинаково (то есть, logreg и logreg__C (2 подчеркивания))

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_C = grid_search.best_params_['logreg__C']
print(best_C)

y_test_pred = grid_search.predict(X_test)

test_accuracy = accuracy_score(y_test, y_test_pred)
print(test_accuracy)

0.0001
0.8076923076923077


In [22]:
taxi = pd.read_csv('/Users/olgazdanova/Downloads/taxi.csv')
taxi.head()

,trip_duration,distance_traveled,num_of_passengers,fare,miscellaneous_fees,total_fare,surge_applied,tip_class
0,748.0,2.75,1.0,75.00,6.300,105.300,0,over med
1,1187.0,3.43,1.0,105.00,13.200,142.200,0,over med
2,329.0,2.09,1.0,45.00,13.200,70.200,0,under med
3,355.0,1.74,1.0,45.00,26.625,71.625,1,low
4,1288.0,5.21,1.0,116.25,26.325,178.575,1,high


In [23]:
X = taxi.drop('tip_class', axis=1)
y = taxi['tip_class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [26]:
# проверяем разный ли масштаб у данных и нужен ли standardscaler 
# масштаб разный
X_train.head()

,trip_duration,distance_traveled,num_of_passengers,fare,miscellaneous_fees,total_fare,surge_applied
58887,414.0,1.29,1.0,45.00,13.500,58.500,0
106949,631.0,1.93,1.0,60.00,6.000,66.000,0
73981,1242.0,5.83,1.0,112.50,26.475,159.975,1
96565,609.0,1.35,1.0,56.25,26.125,120.375,1
5738,1539.0,12.26,1.0,195.00,26.625,221.625,1


In [27]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['float64', 'int64']))
X_test_scaled = scaler.transform(X_test.select_dtypes(include=['float64', 'int64'])) # тут убираем fit, потому что обучается только на train данных

In [28]:
# обучаем модель уже на масштабируемых данных и смотрим метрику
model = LogisticRegression(max_iter=400)
model.fit(X_train_scaled, y_train)

y_train_pred = model.predict(X_train_scaled)
y_test_pred = model.predict(X_test_scaled)

print(accuracy_score(y_train, y_train_pred))
print(accuracy_score(y_test, y_test_pred))

0.9600572303430175
0.9594648780665622


In [29]:
# посчитайте, сколько получилось ошибочных предсказаний в процентном соотношении по каждому классу на тестовой выборке
from sklearn.metrics import confusion_matrix
import numpy as np

conf_matrix = confusion_matrix(y_test, y_test_pred)

classes = np.unique(y_test)

# Расчет процента ошибочных предсказаний по каждому классу
error_percentages = {}
for i, class_name in enumerate(classes):
    total_samples = np.sum(conf_matrix[i, :])  # Общее количество примеров класса
    incorrect_predictions = total_samples - conf_matrix[i, i]  # Количество ошибок
    error_percentage = (incorrect_predictions / total_samples) * 100  # Процент ошибок
    error_percentages[class_name] = round(error_percentage, 2)  # Округление до 2 знаков
    
print("Процент ошибочных предсказаний по каждому классу:")
for class_name, error_percentage in error_percentages.items():
    print(f"{class_name}: {error_percentage}%")
    
# low, over med неправильно

Процент ошибочных предсказаний по каждому классу:
high: 2.46%
low: 7.07%
over med: 6.87%
under med: 0.03%
